In [2]:
import os
from dotenv import load_dotenv
import gradio as gr
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# LangChain imports
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.tools import Tool
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [4]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "Medicines"

In [6]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [8]:
# --- Database Connection Details ---
SERVER_NAME = "localhost"
DATABASE_NAME = "ChatbotFarmacia" # <-- Good, you've set your DB name
TABLE_NAME = "Medicines" # Note: TABLE_NAME here isn't used for the engine itself
SCHEMA_NAME = "dbo"      # Note: SCHEMA_NAME here isn't used for the engine itself
driver = "ODBC Driver 17 for SQL Server"
connection_string = f"mssql+pyodbc://{SERVER_NAME}/{DATABASE_NAME}?driver={driver}&trusted_connection=yes"


# --- Create Engine & Load Data ---
df = pd.DataFrame()
chunks = []
try:
    print(f"Connecting to DB: {DATABASE_NAME} on {SERVER_NAME}...")
    engine = create_engine(connection_string)
    sql_query = f"SELECT * FROM [dbo].[Medicines]" # Or your relevant query
    print(f"Loading data...")
    df = pd.read_sql(sql_query, engine)
    print(f"Successfully loaded {len(df)} rows.")

    # --- Split DataFrame into Chunks ---
    # This line creates the 'chunks' variable needed below
    chunks = [df.iloc[i:i+5] for i in range(0, len(df), 5)]
    print(f"Data split into {len(chunks)} chunks.")

except Exception as e:
    print(f"Error loading data or creating chunks: {e}")
# --- Create Database Engine ---
try:
    print(f"Attempting to connect to {DATABASE_NAME} on {SERVER_NAME}...")
    engine = create_engine(connection_string)
    # Optional connection test
    connection = engine.connect()
    print(f"Successfully connected to database '{DATABASE_NAME}' on '{SERVER_NAME}'.")
    connection.close()
    print("SQLAlchemy engine created.")

except Exception as e:
    print(f"Error connecting to database: {e}")
    # Handle error
    exit()

# --- The 'engine' variable created above is what you need for the SQL Agent ---

include_tables = ["Medicines", "inventory", "inventory_chorrera", "inventory_costa_del_este", "inventory_david", "inventory_el_dorado", "inventory_san_francisco",  "Stores"] # List all tables
db = SQLDatabase(engine=engine, schema="dbo", include_tables=include_tables)
# Optional: print(db.get_table_info())

Connecting to DB: ChatbotFarmacia on localhost...
Loading data...
Successfully loaded 627 rows.
Data split into 126 chunks.
Attempting to connect to ChatbotFarmacia on localhost...
Successfully connected to database 'ChatbotFarmacia' on 'localhost'.
SQLAlchemy engine created.


In [10]:
llm = ChatOpenAI(model=MODEL, temperature=2) # Low temp recommended for agent logic

# Add this import line, typically near the top with your other imports


# --- Your existing code ---
# llm = ChatOpenAI(...)
# db = SQLDatabase(...)
# --- End of existing code ---

# Now you can create the agent (this line should work after the import)
sql_agent = create_sql_agent(
    llm=llm, 
    db=db, 
    agent_type="openai-tools", 
    verbose=True,
    prefix="""You are an expert SQL agent for a pharmacy system. 
    
    You have access to tables including 'Stores' which contains store location information. 
    
    When asked about stores, store counts, or locations, always query the Stores table.
    When asked "how many stores", run 'SELECT COUNT(*) FROM dbo.Stores'.
    
    Always check the schema carefully before answering and provide clear, concise responses.
    """
)
print("SQL Agent created successfully.")

SQL Agent created successfully.


In [12]:
# --- Load the DataFrame (ensure this is done before running the agent) ---
try:
    # Define the SQL query to fetch data from the "Stores" table
    query = "SELECT StoreID, StoreName, Location FROM dbo.Stores" # Select only needed columns

    # Execute the query and load the result into a DataFrame
    # Ensure 'engine' is correctly initialized with your DB connection
    stores_df = pd.read_sql(query, engine)
    print(f"Successfully loaded {len(stores_df)} stores into DataFrame.")
    # Keep only the DataFrame in memory, maybe close the engine if not needed elsewhere
    # engine.dispose()
except Exception as e:
    print(f"Error loading stores data: {e}")
    # Handle the error appropriately, maybe exit or use dummy data
    stores_df = pd.DataFrame() # Create an empty DataFrame to prevent errors later

# --- Make sure stores_df is accessible globally or passed correctly ---

Successfully loaded 5 stores into DataFrame.


In [14]:
# Simple LangChain implementation without typing

# Basic imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import tool
from langchain.memory import ChatMessageHistory

# Session storage for maintaining conversation history
session_histories = {}

# Define the models/LLMs
# Assuming MODEL is defined elsewhere in your code
llm = ChatOpenAI(model=MODEL, temperature=0.7)
agent_llm = ChatOpenAI(model=MODEL, temperature=0)

# Load the vectorstore
if 'vectorstore' not in locals() or vectorstore is None:
    print("Loading existing vectorstore from disk...")
    embeddings = OpenAIEmbeddings()
    try:
        vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
        print(f"Loaded vectorstore with {vectorstore._collection.count()} documents")
    except Exception as e:
        print(f"Error loading vectorstore: {e}")

# Set up the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Function to get or create session history
def get_session_history(session_id):
    if session_id not in session_histories:
        session_histories[session_id] = ChatMessageHistory()
    return session_histories[session_id]

# Create a RAG chain
def create_rag_chain():
    # Define the prompt template
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""You are an expert pharmaceutical assistant. Use the following context to answer the question.
        
If you're asked about side effects, focus on the information in the 'Side Effects (Common)' and 'Side Effects (Rare)' fields.
If you're asked about stores or inventory, explain that this information needs to be queried from the database.
Answer the question based only on the provided context. If the information isn't available, say so clearly."""),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessage(content="{question}"),
        SystemMessage(content="Context: {context}")
    ])
    
    # Create the RAG chain
    return (
        {"context": retriever, "question": RunnablePassthrough(), "chat_history": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# Create the RAG chain
rag_chain = create_rag_chain()

# Define the SQL query tool
@tool
def sql_query(query):
    """Execute a SQL query against the store and inventory database"""
    try:
        # Assuming sql_agent is defined elsewhere
        return sql_agent.invoke({"input": query})["output"]
    except Exception as e:
        return f"Error querying database: {str(e)}"

# Vector search function for direct access to RAG
def vector_search(query, session_id="default"):
    try:
        # Get history and pass it explicitly
        history = get_session_history(session_id)
        result = rag_chain.invoke({"question": query, "chat_history": history.messages})
        
        # Record the exchange
        history.add_user_message(query)
        history.add_ai_message(result)
        return result
    except Exception as e:
        print(f"Error in vector_search: {e}")
        # Fallback to basic query without history
        return rag_chain.invoke({"question": query, "chat_history": []})

# Simple agent without LangGraph
def simple_agent(query):
    """A simple agent implementation that doesn't use LangGraph"""
    # Create a prompt for the agent
    agent_prompt = f"""You are a helpful assistant that can answer questions about medicines and store inventory.

Question: {query}

If this is about store inventory, locations, or similar store-related information, use the SQL database.
If this is about medicine properties, side effects, or drug information, use the medicine information database.
Otherwise, answer directly.

Respond with your final answer."""

    # Get a response from the LLM
    response = agent_llm.invoke(agent_prompt)
    
    # Extract the content
    if hasattr(response, "content"):
        return response.content
    else:
        return str(response)

# The integrated chat function
def chat(question, session_id="default"):
    try:
        # For store-related questions, directly route to SQL agent
        if any(keyword in question.lower() for keyword in ["store", "stores", "location", "locations", "inventory", "stock", "how many"]):
            try:
                print(f"Routing to SQL agent: {question}")
                result = sql_query(question)
                
                # Record the exchange in history
                history = get_session_history(session_id)
                history.add_user_message(question)
                history.add_ai_message(result)
                return result
            except Exception as e:
                print(f"SQL direct routing failed: {e}, falling back to simple agent")
        
        # For medicine-related questions about side effects, use RAG
        if any(keyword in question.lower() for keyword in ["side effect", "medicine", "drug", "medication"]):
            try:
                print(f"Routing to RAG chain: {question}")
                return vector_search(question, session_id)
            except Exception as e:
                print(f"RAG chain failed: {e}, falling back to simple agent")
        
        # For general questions, use the simple agent
        try:
            print(f"Using simple agent: {question}")
            result = simple_agent(question)
            
            # Record the exchange in history
            history = get_session_history(session_id)
            history.add_user_message(question)
            history.add_ai_message(result)
            
            return result
        except Exception as e:
            print(f"Agent failed: {e}, falling back to direct LLM")
            try:
                response = llm.invoke(question)
                content = response.content if hasattr(response, "content") else str(response)
                return content
            except Exception as llm_err:
                return f"I encountered several errors processing your request. Please try rephrasing your question. Error: {str(llm_err)}"
                
    except Exception as e:
        return f"I encountered an error: {str(e)}. Please try rephrasing your question."

Loading existing vectorstore from disk...
Loaded vectorstore with 1254 documents


In [15]:

def get_store_count() -> str:
  """
  Use this tool ONLY when asked about the total number, count, quantity, or amount of store locations the company has.
  Returns the total count as a string.
  """
  global stores_df # Access the DataFrame (or pass it in if preferred)
  if stores_df is None or stores_df.empty:
      return "I cannot access the store data right now to determine the count."
  num_stores = len(stores_df)
  return f"There are currently {num_stores} store locations."

# Create a list of tools for the agent
tools = [get_store_count]

In [18]:

# Define the SQL query to fetch data from the "Stores" table
query = "SELECT * FROM dbo.Stores"

# Execute the query and load the result into a DataFrame
stores_df = pd.read_sql(query, engine)
print(stores_df.head())  # Print the first few rows to inspect the data



   StoreID       StoreName        InventoryTableName  \
0        1        Chorrera        inventory_chorrera   
1        2  Costa del Este  inventory_costa_del_este   
2        3           David           inventory_david   
3        4       El Dorado       inventory_el_dorado   
4        5   San Francisco   inventory_san_francisco   

                       Location  
0    Panamá Oeste - La Chorrera  
1  Panama City - Costa del Este  
2              Chiriquí - David  
3       Panama City - El Dorado  
4   Panama City - San Francisco  


In [20]:
# --- Corrected Document Creation Loop (Option 1) ---
# Assuming 'chunks' is your list of DataFrames from the SQL query
# Requires: from langchain.docstore.document import Document

docs = []
print("Starting document conversion (translating 1/0 status to text in page_content)...")
for i, chunk_df in enumerate(chunks):
    for index, row in chunk_df.iterrows():
        try:
            # --- Get the numeric status (assuming column name is 'Prescription') ---
            try:
                 # Use the actual column name from your SQL table if different from 'Prescription'
                 status_flag = int(row.get('Prescription', -1)) # Get 1, 0, or -1
            except (ValueError, TypeError):
                 status_flag = -1 # Handle non-numeric or missing data

            # --- Translate numeric status to text ---
            if status_flag == 1:
                status_text = "Requires Prescription"
            elif status_flag == 0:
                status_text = "Over-the-Counter"
            else:
                status_text = "Unknown"

            # --- MODIFIED page_content to include the status TEXT ---
            # Use correct column names from your SQL table (e.g., 'Generic Name', 'Uses')
            page_content = f"Medicine: {row['Generic Name']}\nUses: {row['Uses']}\nPrescription Status: {status_text}"

            # --- Metadata: Store the numeric flag and other relevant fields ---
            metadata = {
                "source_db_table": f"{SCHEMA_NAME}.{TABLE_NAME}", # Identify source
                # Use primary key from DB if available and useful, otherwise use index
                # "db_primary_key": row.get('YourPrimaryKeyColumn'),
                "chunk_index": i,
                # Store the numeric flag using a clear key name
                "prescription_required_flag": status_flag,
                # Add other relevant fields from your DB table, ensuring column names match
                "uses": row.get('Uses', ""),
                "side_effects_common": row.get('Side Effects (Common)', ""),
                "side_effects_rare": row.get('Side Effects (Rare)', ""),
                "similar_drugs": row.get('Similar Drugs', ""),
                "brand_name_1": row.get('Brand Name 1', ""),
                # ... etc
            }
            docs.append(Document(page_content=page_content, metadata=metadata))
        except KeyError as e:
            print(f"KeyError processing row {index}: {e} - Check column names from DB query!")
        except Exception as e:
             print(f"Error processing row {index}: {e}")

print(f"Created {len(docs)} Document objects with updated page_content.")
# --- End of Corrected Document Creation Loop ---

Starting document conversion (translating 1/0 status to text in page_content)...
Created 627 Document objects with updated page_content.


In [22]:
query = "How many stores are there?"
result = sql_agent.invoke({"input": query})
print(result.get("output", "No output provided."))



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Medicines, Stores, inventory, inventory_chorrera, inventory_costa_del_este, inventory_david, inventory_el_dorado, inventory_san_francisco
Invoking: `sql_db_schema` with `{'table_names': 'Stores'}`



CREATE TABLE dbo.[Stores] (
	[StoreID] INTEGER NOT NULL IDENTITY(1,1), 
	[StoreName] NVARCHAR(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[InventoryTableName] NVARCHAR(128) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Location] NVARCHAR(255) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__Stores__3B82F0E1F0128E59] PRIMARY KEY ([StoreID])
)

/*
3 rows from Stores table:
StoreID	StoreName	InventoryTableName	Location
1	Chorrera	inventory_chorrera	Panamá Oeste - La Chorrera
2	Costa del Este	inventory_costa_del_este	Panama City - Costa del Este
3	David	inventory_david	Chiriquí - David
*/
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM dbo.Stores'}`


[(5,)]There are 5 

In [23]:
# Optionally, view the first chunk
print(len(chunks))

126


In [ ]:
def get_store_count() -> str:
  """
  Use this tool ONLY when asked about the total number, count, quantity, or amount of store locations the company has.
  Returns the total count as a string.
  """
  global stores_df # Access the DataFrame (or pass it in if preferred)
  if stores_df is None or stores_df.empty:
      return "I cannot access the store data right now to determine the count."
  num_stores = len(stores_df)
  return f"There are currently {num_stores} store locations."

# Create a list of tools for the agent
tools = [get_store_count]

In [26]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

embeddings = OpenAIEmbeddings() # Assumes 'from langchain_openai import OpenAIEmbeddings' was used
                               # and the OpenAI API key is configured (e.g., environment variable)

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists
# Assumes 'db_name' variable (string path) is defined earlier
# Assumes 'import os' and 'from langchain_chroma import Chroma' were used

if os.path.exists(db_name):
    try:
        # Attempt to connect and delete the collection within the directory
        Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
        print(f"Deleted existing collection in '{db_name}'.")
    except Exception as e:
        # Handle cases where deletion might fail (e.g., directory exists but isn't a valid Chroma DB)
        print(f"Could not delete collection in '{db_name}': {e}")

# Create vectorstore

vectorstore = Chroma.from_documents(documents=docs, # 'docs' needs to be List[Document]
                                     embedding=embeddings,
                                     persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents in '{db_name}'.")

Deleted existing collection in 'Medicines'.
Vectorstore created with 627 documents in 'Medicines'.


In [27]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 1254 documents


In [30]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 1,254 vectors with 1,536 dimensions in the vector store


In [32]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings']) # Requires: import numpy as np
documents = result['documents']
metadatas = result['metadatas']
# doc_types = [metadata['doc_type'] for metadata in metadatas if metadata is not None] # REMOVED/COMMENTED

# You can now work with vectors, documents, metadatas
print(f"Retrieved {len(vectors)} items.")
if metadatas:
     print("First item metadata:", metadatas[0])

Retrieved 1254 items.
First item metadata: {'brand_name_1': 'Nulojix', 'chunk_index': 0, 'prescription_required_flag': 1, 'side_effects_common': 'Hypertension, diarrhea, anemia', 'side_effects_rare': 'Post-transplant lymphoproliferative disorder, infections', 'similar_drugs': 'Basiliximab, Tacrolimus', 'source_db_table': 'dbo.Medicines', 'uses': 'Prevention of kidney transplant rejection'}


In [34]:
# Define the model name as a string
MODEL = "gpt-4"

# Import the modern components
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create a Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)
# Alternative - if you'd like to use Ollama locally, uncomment this line instead
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# The retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# Create a message history store
message_histories = {}

# Function to get or create a message history for a session
def get_message_history(session_id):
    if session_id not in message_histories:
        message_histories[session_id] = ChatMessageHistory()
    return message_histories[session_id]

# Create the modern conversation chain
def create_conversation_chain():
    # Define the prompt
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are a helpful assistant. Answer based on the retrieved context."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessage(content="{question}"),
        SystemMessage(content="Context: {context}")
    ])
    
    # Create the chain
    chain = (
        {"context": retriever, "question": lambda x: x["question"]}
        | prompt
        | llm
    )
    
    # Wrap with history management
    return RunnableWithMessageHistory(
        chain,
        get_message_history,
        input_messages_key="question",
        history_messages_key="chat_history"
    )

# Create the conversation chain
conversation_chain = create_conversation_chain()

# Function to query the chain
def query(question, session_id="default"):
    return conversation_chain.invoke(
        {"question": question},
        {"configurable": {"session_id": session_id}}
    )

In [36]:
# Set up a simpler implementation focused on clarity
from langchain_core.callbacks import StdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Create a callback handler
handler = StdOutCallbackHandler()

# Initialize the LLM with callbacks
llm = ChatOpenAI(
    temperature=0.7, 
    model_name=MODEL, 
    callbacks=[handler]
)

# Function to directly query about headache medications
def direct_query(question):
    # Get documents from retriever
    docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Create a simple, explicit prompt
    messages = [
        SystemMessage(content="You are a helpful assistant that answers questions about medications."),
        HumanMessage(content=f"""
I need information about medications for headaches.

Here is my question: {question}

Here is information from our medical database to help you answer:
{context}

Please provide a comprehensive answer based on this information.
""")
    ]
    
    # Get response from LLM
    response = llm.invoke(messages)
    return response.content

# Execute the query
query = "What medicines are good for headaches?"
print("Sending query:", query)
result = direct_query(query)
print("\nAnswer:", result)

Sending query: What medicines are good for headaches?

Answer: There are several medications that can be effective for treating headaches, depending on the type and severity. 

Butalbital is one medication that is often used for tension headaches and as part of combination therapy for migraines. Please note that Butalbital requires a prescription from a healthcare provider.

Another medication is Sumatriptan, which is used for the acute treatment of migraines and cluster headaches. Like Butalbital, Sumatriptan also requires a prescription.

It's important to discuss with your healthcare provider about these medications, as they can advise you based on your specific symptoms and overall health condition.


In [37]:
# Import the new memory class
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Create base conversation chain without memory first
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever
)

# Set up message history store
message_history = ChatMessageHistory()

# Wrap the chain with message history capability
conversation_with_memory = RunnableWithMessageHistory(
    conversation_chain,
    lambda session_id: message_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

In [38]:
def chat(question, history):
    print("Using SQL agent-enabled chat function")  # Debug print
    try:
        response = run_sql_agent(question, sql_agent)
        history = history + [(question, response)]
        return response, history
    except Exception as e:
        error_msg = f"Error during SQL agent invocation: {str(e)}"
        print(error_msg)
        return error_msg, history



In [40]:
# Define your SQL agent and run_sql_agent before launching the interface
def run_sql_agent(query, sql_agent):
    try:
        result = sql_agent.invoke({"input": query})
        output = result.get("output", "No output provided.")
        return output
    except Exception as e:
        return f"Error querying database: {str(e)}"

def chat(question, history):
    print("Using SQL agent-enabled chat function")
    try:
        print(f"Sending question to SQL agent: {question}")
        response = run_sql_agent(question, sql_agent)
        print(f"Received response: {response[:100]}...")  # Print first 100 chars
        history = history + [(question, response)]
        return response, history
    except Exception as e:
        error_msg = f"Error during SQL agent invocation: {str(e)}"
        print(error_msg)
        import traceback
        print(traceback.format_exc())  # Print full traceback
        return error_msg, history

In [44]:
query = "How many stores are there?"
response = run_sql_agent(query, sql_agent)
print(response)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Medicines, Stores, inventory, inventory_chorrera, inventory_costa_del_este, inventory_david, inventory_el_dorado, inventory_san_francisco
Invoking: `sql_db_schema` with `{'table_names': 'Stores'}`



CREATE TABLE dbo.[Stores] (
	[StoreID] INTEGER NOT NULL IDENTITY(1,1), 
	[StoreName] NVARCHAR(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[InventoryTableName] NVARCHAR(128) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Location] NVARCHAR(255) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__Stores__3B82F0E1F0128E59] PRIMARY KEY ([StoreID])
)

/*
3 rows from Stores table:
StoreID	StoreName	InventoryTableName	Location
1	Chorrera	inventory_chorrera	Panamá Oeste - La Chorrera
2	Costa del Este	inventory_costa_del_este	Panama City - Costa del Este
3	David	inventory_david	Chiriquí - David
*/
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM dbo.Stores'}`


[(5,)]There are a 

In [46]:
# --- The Integrated Chat Function (from farmabot.txt) ---
# Ensure llm, sql_agent, and conversation_chain are defined and initialized correctly before this function
def chat(message_list):
    """
    Handles a list of message dictionaries and routes to appropriate system:
    - SQL queries go to the SQL agent
    - Medication questions go to the RAG system
    - Other queries get a direct response

    Args:
        message_list: List of message dictionaries with 'role' and 'content' keys [cite: 58]

    Returns:
        String response to the user
    """
    print("Received message list:", message_list) # Debug print [cite: 58]

    if not message_list: # [cite: 58]
        return "Say something!" # [cite: 58]

    # Get the content of the last message (which should be from the user)
    last_message = message_list[-1] # [cite: 59]
    user_content = last_message.get('content', '') # Use .get for safety [cite: 59]

    # Store previous messages to build conversation context
    previous_messages = [] # [cite: 59]
    for msg in message_list[:-1]: # All messages except the last one [cite: 59]
        if msg.get('role') in ['user', 'assistant']: # [cite: 59]
            previous_messages.append(f"{msg.get('role')}: {msg.get('content', '')}") # [cite: 59]

    conversation_context = "\n".join(previous_messages) # [cite: 59]
    print(f"Conversation context length: {len(conversation_context)} characters") # [cite: 59]

    # Determine which system to use based on query content
    # 1. SQL-related keywords [cite: 60]
    sql_keywords = ['store', 'inventory', 'location', 'how many', 'where', 'stock', # [cite: 60]
                   'database', 'query', 'find stores', 'nearest', 'available'] # [cite: 60]

    # 2. Medicine-related keywords [cite: 60]
    medicine_keywords = ['medicine', 'drug', 'medication', 'prescription', 'side effect', # [cite: 60]
                        'dosage', 'treatment', 'headache', 'pain', 'symptom'] # [cite: 60]

    # Check if the query matches SQL patterns
    if any(keyword in user_content.lower() for keyword in sql_keywords): # [cite: 61]
        print(f"Routing to SQL agent: {user_content}") # [cite: 61]
        try:
            # Assuming sql_agent is defined elsewhere and initialized
            result = sql_agent.invoke({"input": user_content}) # [cite: 61]
            response = result.get("output", "I couldn't find that information in our store database.") # [cite: 61]
            print(f"SQL response: {response[:100]}...") # Print first 100 chars [cite: 62]
            return response # [cite: 62]
        except Exception as e: # [cite: 62]
            error_msg = f"I encountered an error querying the store database: {str(e)}" # [cite: 62]
            print(error_msg) # [cite: 62]
            # Fall back to RAG if SQL fails
            print("Falling back to general knowledge...") # [cite: 63]

    # Check if the query matches medicine patterns
    if any(keyword in user_content.lower() for keyword in medicine_keywords): # [cite: 63]
        print(f"Routing to RAG system: {user_content}") # [cite: 63]
        try:
            # Assuming conversation_chain is your RAG chain and initialized
            # Note: The original code used conversation_chain.invoke({"question": user_content})
            # Adjust based on how your specific RAG chain expects input (it might need session_id management)
            # For simplicity here, we'll assume it takes just the question. Modify if needed.
            # Using vector_search function defined earlier which handles history might be better if applicable
            result = conversation_chain.invoke({"question": user_content}) # Check if this matches your RAG chain's input [cite: 63]
            # Extract the answer based on your RAG chain's output structure
            # Example: response = result.get("answer", "Default message") or result['answer'] or result.content
            # Using the structure from the source file:
            response = result.get("answer", "I couldn't find information about that medication.") # [cite: 64]
            print(f"RAG response: {response[:100]}...") # Print first 100 chars [cite: 64]
            return response # [cite: 64]
        except Exception as e: # [cite: 64]
            error_msg = f"I encountered an error retrieving medication information: {str(e)}" # [cite: 64]
            print(error_msg) # [cite: 64]
            # Fall back to direct response [cite: 65]

    # Default response for general queries
    print("Using default response generation") # [cite: 65]
    try:
        # Use the LLM directly for general conversation
        from langchain_core.messages import HumanMessage, SystemMessage # [cite: 65]

        messages = [ # [cite: 65]
            SystemMessage(content=f"""You are a helpful pharmacy assistant.
            Be conversational and friendly. # [cite: 66]

            Previous conversation:
            {conversation_context}"""), # [cite: 66]
            HumanMessage(content=user_content) # [cite: 66]
        ]

        # Assuming llm is defined elsewhere and initialized
        response = llm.invoke(messages).content # [cite: 66]
        return response # [cite: 66]
    except Exception as e: # [cite: 66]
        # Ultimate fallback
        return f"I understand you said: '{user_content}', but I'm having trouble generating a response right now. How else can I help you?" # [cite: 67, 68]





In [ ]:
import gradio as gr

# Wrap your integrated chat function in a Gradio-friendly format.
def gradio_chat(message, history):
    # If history is empty, initialize it as a list
    if history is None:
        history = []
    
    # Append the new user message to the conversation history.
    history.append({"role": "user", "content": message})
    
    # Call the integrated chat function.
    # This function should use the full message list to generate a response
    # based on your SQL and RAG routing logic.
    response = chat(history)
    
    # Append the assistant response to the conversation history.
    history.append({"role": "assistant", "content": response})
    
    # Return the updated conversation history and clear the input.
    return history, ""

# Create the Gradio interface using Blocks.
with gr.Blocks() as demo:
    gr.Markdown("# Pharmacy Assistant Chat")
    
    # Chatbot component displays conversation as a list of message dictionaries.
    chatbot = gr.Chatbot(elem_id="chatbot")
    
    # Input textbox for user messages.
    msg = gr.Textbox(
        placeholder="Type your message here...", 
        label="Your Message"
    )
    
    # A button to send messages.
    send = gr.Button("Send")
    
    # Set up event handling: on button click or enter key press.
    send.click(fn=gradio_chat, inputs=[msg, chatbot], outputs=[chatbot, msg])
    msg.submit(fn=gradio_chat, inputs=[msg, chatbot], outputs=[chatbot, msg])

# Launch the interface.
demo.launch(debug=True)


In [ ]:
# --- Gradio Interface Definition ---
def create_interface():
    """Creates the Gradio interface"""

    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("# Farma AI Panama") # Title for the interface

        # Chatbot component to display messages
        chatbot = gr.Chatbot(
            height=500,
            type="messages" # Use "messages" format for role support [cite: 70]
        )

        # Row for input textbox and send button
        with gr.Row(): # [cite: 70]
            msg = gr.Textbox(
                placeholder="Ask about medicines, inventory, or store locations...", # [cite: 70]
                show_label=False, # [cite: 70]
                scale=4 # Make textbox wider [cite: 70]
            )
            submit_button = gr.Button("Send", variant="primary", scale=1) # [cite: 71]

        # Function to handle message submission and update chatbot
        def handle_submit(user_message, chat_history):
            """
            Takes user message and history, calls the main chat function,
            and returns the updated history.
            """
            # Append the user message to the history in the expected format
            chat_history.append({"role": "user", "content": user_message})

            # Call your main chat function which expects a list of message dicts
            response = chat(chat_history)

            # Append the assistant's response
            chat_history.append({"role": "assistant", "content": response})

            # Return None to clear the textbox, and the updated history for the chatbot
            return "", chat_history

        # Event handlers
        submit_button.click(
            fn=handle_submit,
            inputs=[msg, chatbot],
            outputs=[msg, chatbot] # Clear msg textbox and update chatbot
        ) # [cite: 71]

        msg.submit(
            fn=handle_submit,
            inputs=[msg, chatbot],
            outputs=[msg, chatbot] # Clear msg textbox and update chatbot
        ) # [cite: 72, 73]

    return demo

# --- Launch the Interface ---
if __name__ == "__main__":
    # --- Ensure all setup code from farmabot.txt runs before this ---
    # (Database connection, LLM/Agent/Retriever initialization, etc.)
    print("Creating and launching Gradio interface...")
    interface = create_interface()
    # Add share=True here:
    interface.launch(debug=True, share=True) # share=True generates a public link
    print("Gradio interface launched. Check the output for the public URL.")

Creating and launching Gradio interface...
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://976b762e574190deb9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Received message list: [{'role': 'user', 'content': 'Do you have cocaine '}]
Conversation context length: 0 characters
Using default response generation
Received message list: [{'role': 'user', 'metadata': None, 'content': 'Do you have cocaine ', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "I'm sorry, but as a pharmacy assistant, I must clarify that cocaine is an illegal substance and it is not sold or distributed in any legal pharmacies. It's important to only use medications prescribed by a healthcare professional. Please let me know if you need any help with legal, over-the-counter or prescription medications.", 'options': None}, {'role': 'user', 'content': 'What is the recommended dose of Ritalin for a 40 year old male?'}]
Conversation context length: 366 characters
Using default response generation
Received message list: [{'role': 'user', 'metadata': None, 'content': 'Do you have cocaine ', 'options': None}, {'role': 'assistant', 'metadata': None, 'content